In [1]:
import torch

if torch.cuda.is_available():
    print(f"✅ Running on GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ Running on CPU")

✅ Running on GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import torch
import torch.nn as nn

In [3]:
class bottleneck (nn.Module):
  def __init__(self, input_filter, output_filter, stride_value):
    super().__init__()
    self.conv1 = nn.Conv2d(input_filter, output_filter, kernel_size=1, stride = 1, padding=0)
    self.bn1 = nn.BatchNorm2d(output_filter)
    self.conv2 = nn.Conv2d(output_filter, output_filter, kernel_size=3, stride = stride_value, padding=1)
    self.bn2 = nn.BatchNorm2d(output_filter)
    self.conv3 = nn.Conv2d(output_filter, output_filter*4, kernel_size=1, stride = 1, padding=0)
    self.bn3 = nn.BatchNorm2d(output_filter*4)
    self.relu = nn.ReLU()

    if input_filter != output_filter * 4 or stride_value != 1:
      self.downsample = nn.Sequential(
          nn.Conv2d(input_filter, output_filter * 4, kernel_size=1, stride=stride_value, padding=0),
          nn.BatchNorm2d(output_filter * 4))
    else:
      self.downsample = None


  def forward(self, x):
    start = x
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)
    out = self.conv3(out)
    out = self.bn3(out)

    if self.downsample is not None:
      start = self.downsample(start)


    out = out + start
    out = self.relu(out)
    return out

In [4]:
class MyResnet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
    self.mpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()

    #ResNet stages
    self.layer1 = nn.Sequential(
        bottleneck(64, 64, 1),
        bottleneck(256, 64, 1),
        bottleneck(256, 64, 1)
    )
    self.layer2 = nn.Sequential(
        bottleneck(256, 128, 2),
        bottleneck(512, 128, 1),
        bottleneck(512, 128, 1),
        bottleneck(512, 128, 1)
    )
    self.layer3 = nn.Sequential(
        bottleneck(512, 256, 2),
        bottleneck(1024, 256, 1),
        bottleneck(1024, 256, 1),
        bottleneck(1024, 256, 1),
        bottleneck(1024, 256, 1),
        bottleneck(1024, 256, 1)
    )
    self.layer4 = nn.Sequential(
        bottleneck(1024, 512, 2),
        bottleneck(2048, 512, 1),
        bottleneck(2048, 512, 1)
    )

    #final average pooling
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(2048, 1000)

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.mpool1(out)

    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)

    out = self.avgpool(out)
    out = torch.flatten(out, 1)
    out = self.fc(out)
    return out

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# ======================
# 1. Data Preparation
# ======================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects large images
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                           shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                          shuffle=False, num_workers=2)

# ======================
# 2. Model Initialization
# ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyResnet()
model.fc = nn.Linear(2048, 10)  # CIFAR-10 has 10 classes
model = model.to(device)

# ======================
# 3. Loss & Optimizer
# ======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ======================
# 4. Training Loop
# ======================
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(trainloader):.4f}")

# ======================
# 5. Evaluation
# ======================
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/5], Loss: 1.7184
Epoch [2/5], Loss: 1.2258
Epoch [3/5], Loss: 0.9734
Epoch [4/5], Loss: 0.7965
Epoch [5/5], Loss: 0.6796
Accuracy on test set: 72.90%
